In [1]:
from bs4 import BeautifulSoup
import pandas as pd
import requests

In [2]:
file = "NPX_data1.xlsx"
df = pd.read_excel(file, skiprows=1)
df_indexed = df.set_index("NPX Data")
df_cleaned = df_indexed.iloc[:, 0:1104]
col_len = len(df_cleaned.columns)
for x in range(col_len):
    df_cleaned.iloc[0][x] = df_cleaned.iloc[0][x].split("Olink ")[1]

In [4]:
protein_data = {
    "Panel" : [],
    "Uniprot ID": []
}
for y in range(col_len):
    panel = df_cleaned.iloc[[0, 2], y][0]
    uniprot = df_cleaned.iloc[[0, 2], y][1]
    try:
        if len(uniprot.split(",")) > 1:
            uniprot_split = uniprot.split(",")
            protein_data["Panel"].append(panel)
            protein_data["Uniprot ID"].append(uniprot_split[0])
            protein_data["Panel"].append(panel)
            protein_data["Uniprot ID"].append(uniprot_split[1])
        elif len(uniprot.split("-")) > 1:
            uniprot_split = uniprot.split("-")
            protein_data["Panel"].append(panel)
            protein_data["Uniprot ID"].append(uniprot_split[0])
        else:
            protein_data["Panel"].append(panel)
            protein_data["Uniprot ID"].append(uniprot)
    except AttributeError:
        protein_data["Panel"].append(panel)
        protein_data["Uniprot ID"].append(uniprot)

In [5]:
panel_uniprot_df = pd.DataFrame(protein_data)
panel_uniprot_df = panel_uniprot_df.dropna()

In [7]:
len(panel_uniprot_df)

1106

In [13]:
panel_uniprot_df.to_csv("panel_uniprot.csv")
uniprot_id = panel_uniprot_df["Uniprot ID"].tolist()
len(uniprot_id)

1106

In [109]:
baseURL = "https://www.uniprot.org/uniprot/"
protein_scraped_info = {
    "Uniprot ID" : [],
    "Protein Name" : [],
    "Gene Name" : [],
    "Function" : []
    }
for z in range(len(uniprot_id)):
    url = f'{baseURL}{uniprot_id[z]}'
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'lxml')
    try: 
        protein_name = soup.find("div", id="content-protein").find("h1").text
        gene = soup.find("div", id="content-gene").find("h2").text
        protein_scraped_info["Protein Name"].append(protein_name)
        protein_scraped_info["Gene Name"].append(gene)
        try:
            function_text = soup.find("div", class_="annotation").find("span").text
            text_split = function_text.split("<p>")[0].split("\r")
            protein_scraped_info["Function"].append(text_split[0])
            protein_scraped_info["Uniprot ID"].append(uniprot_id[z])
        except AttributeError:
            protein_scraped_info["Function"].append("Function Not Listed")
            protein_scraped_info["Uniprot ID"].append(uniprot_id[z])
            print(f'{uniprot_id[z]}: Function Not Listed')
    except AttributeError:
        print(f'{uniprot_id[z]}: page not found')
    

P0CG05: page not found
P35475: Function Not Listed
Q9BQR3: Function Not Listed
Q9UKR0: Function Not Listed
Q6ISS4: Function Not Listed


In [157]:
#P0CG05 is deleted from original df

In [105]:
url = f"https://www.uniprot.org/uniprot/P63313"
response = requests.get(url)
soup = BeautifulSoup(response.text, 'lxml')
protein_name = soup.find("div", id="content-protein").find("h1").text
gene = soup.find("div", id="content-gene").find("h2").text
function_text = soup.find("div", class_="annotation").find("span").text
text_split = function_text.split("<p>")[0].split("\r")

In [106]:
text_split[0]

'Plays an important role in the organization of the cytoskeleton. Binds to and sequesters actin monomers (G actin) and therefore inhibits actin polymerization (By similarity).By similarity'

In [77]:
text_split[0]

'Probably involved in lipid transport. Can bind sphingosine-1-phosphate, myristic acid, palmitic acid and stearic acid, retinol, all-trans-retinoic acid and 9-cis-retinoic acid.2 Publications\r\n                                    '

In [108]:
text_split[0].split("By similarity\r")[0]

'Receptor involved in the development of the cardiovascular system, in angiogenesis, in the formation of certain neuronal circuits and in organogenesis outside the nervous system. It mediates the chemorepulsant activity of semaphorins (PubMed:9288753, PubMed:9529250, PubMed:10688880). It binds to semaphorin 3A, The PLGF-2 isoform of PGF, The VEGF165 isoform of VEGFA and VEGFB (PubMed:9288753, PubMed:9529250, PubMed:10688880). Coexpression with KDR results in increased VEGF165 binding to KDR as well as increased chemotaxis. Regulates VEGF-induced angiogenesis. Binding to VEGFA initiates a signaling pathway needed for motor neuron axon guidance and cell body migration, including for the caudal migration of facial motor neurons from rhombomere 4 to rhombomere 6 during embryonic development (By similarity). Regulates mitochondrial iron transport via interaction with ABCB8/MITOSUR (PubMed:30623799).'

In [110]:
sample = pd.DataFrame(protein_scraped_info)
#sample["Protein Name"][22]

'Probably involved in lipid transport. Can bind sphingosine-1-phosphate, myristic acid, palmitic acid and stearic acid, retinol, all-trans-retinoic acid and 9-cis-retinoic acid.2 Publications'

In [113]:
sample = sample.set_index("Uniprot ID")

In [114]:
sample.to_csv("protein_scraped_data.csv")

In [11]:
protein_sc_data = pd.read_csv("FinalProject_LandingPage/ProteinMarkers/analysis/protein_scraped_data.csv")

In [12]:
protein_sc_data

,Uniprot ID,Protein Name,Gene Name,Function
0,O00533,Neural cell adhesion molecule L1-like protein,CHL1,Extracellular matrix and cell adhesion protein...
1,O14786,Neuropilin-1,NRP1,Receptor involved in the development of the ca...
2,O15031,Plexin-B2,PLXNB2,"Cell surface receptor for SEMA4C, SEMA4D and S..."
3,O75015,Low affinity immunoglobulin gamma Fc region re...,FCGR3B,Receptor for the Fc region of immunoglobulins ...
4,O75023,Leukocyte immunoglobulin-like receptor subfami...,LILRB5,May act as receptor for class I MHC antigens.
...,...,...,...,...
1100,P98073,Enteropeptidase,TMPRSS15,Responsible for initiating activation of pancr...
1101,P01229,Lutropin subunit beta,LHB,Promotes spermatogenesis and ovulation by stim...
1102,P53539,Protein fosB,FOSB,FosB interacts with Jun proteins enhancing the...
1103,P27797,Calreticulin,CALR,Calcium-binding chaperone that promotes foldin...
